In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 20.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import shutil
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import cv2
import yaml
import matplotlib.pyplot as plt
from ultralytics import YOLO
import multiprocessing

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
SEED = 3
np.random.seed(SEED)

In [4]:
# Set the data directory
DATA_DIR = Path('/kaggle/input/ghana-crop-disease-detection-data')

# Load train and test files
train = pd.read_csv(DATA_DIR / 'Train.csv')
test = pd.read_csv(DATA_DIR / 'Test.csv')
ss = pd.read_csv(DATA_DIR / 'SampleSubmission.csv')

# Add an image_path column
train['image_path'] = [Path('/kaggle/input/ghana-crop-disease-detection-data/images/' + x) for x in train.Image_ID]
test['image_path'] = [Path('/kaggle/input/ghana-crop-disease-detection-data/images/' + x) for x in test.Image_ID]

# Map string classes to integers (label encoding targets)
train['class_id'] = train['class'].map(
    {'Pepper_Bacterial_Spot': 0, 'Pepper_Fusarium': 1, 'Corn_Cercospora_Leaf_Spot': 2, 'Corn_Common_Rust': 3, 'Tomato_Early_Blight': 4, 'Pepper_Septoria': 5, 'Tomato_Septoria': 6, 'Pepper_Leaf_Curl': 7, 'Pepper_Leaf_Mosaic': 8, 'Corn_Streak': 9, 'Corn_Healthy': 10, 'Pepper_Healthy': 11, 'Tomato_Healthy': 12, 'Pepper_Late_Blight': 13, 'Tomato_Late_Blight': 14, 'Pepper_Cercospora': 15, 'Tomato_Fusarium': 16, 'Pepper_Leaf_Blight': 17, 'Tomato_Leaf_Curl': 18, 'Tomato_Bacterial_Spot': 19, 'Tomato_Mosaic': 20, 'Pepper_Early_Blight': 21, 'Corn_Northern_Leaf_Blight': 22})


In [5]:
# Split data into training and validation
train_unique_imgs_df = train.drop_duplicates(subset=['Image_ID'], ignore_index=True)
X_train, X_val = train_test_split(train_unique_imgs_df, test_size=0.25, stratify=train_unique_imgs_df['class'], random_state=42)

X_train = train[train.Image_ID.isin(X_train.Image_ID)]
X_val = train[train.Image_ID.isin(X_val.Image_ID)]

# Check the shapes of training and validation data
print(X_train.shape, X_val.shape)

(30777, 9) (10252, 9)


In [6]:
# Define directories for images and labels
TRAIN_IMAGES_DIR = Path('/kaggle/working/train/images')
VAL_IMAGES_DIR = Path('/kaggle/working/val/images')
TEST_IMAGES_DIR = Path('/kaggle/working/test/images')
TRAIN_LABELS_DIR = Path('/kaggle/working/train/labels')
VAL_LABELS_DIR = Path('/kaggle/working/val/labels')
TEST_LABELS_DIR = Path('/kaggle/working/test/labels')

In [7]:
# Create a function to setup directories
def setup_directories(dirs):
    for directory in dirs:
        if directory.exists():
            shutil.rmtree(directory)
        directory.mkdir(parents=True, exist_ok=True)

# Parallelize image copying
def copy_image(src_dest_pair):
    src, dest = src_dest_pair
    shutil.copy(src, dest)

In [8]:
# Setup directories for images and labels
setup_directories([TRAIN_IMAGES_DIR, VAL_IMAGES_DIR, TEST_IMAGES_DIR, TRAIN_LABELS_DIR, VAL_LABELS_DIR, TEST_LABELS_DIR])

# Prepare source-destination pairs
train_image_pairs = [(str(img), str(TRAIN_IMAGES_DIR / img.name)) for img in X_train.image_path.unique()]
val_image_pairs = [(str(img), str(VAL_IMAGES_DIR / img.name)) for img in X_val.image_path.unique()]
test_image_pairs = [(str(img), str(TEST_IMAGES_DIR / img.name)) for img in test.image_path.unique()]

# Parallel image copying
with multiprocessing.Pool() as pool:
    list(tqdm(pool.imap(copy_image, train_image_pairs), total=len(train_image_pairs)))
    list(tqdm(pool.imap(copy_image, val_image_pairs), total=len(val_image_pairs)))
    list(tqdm(pool.imap(copy_image, test_image_pairs), total=len(test_image_pairs)))

  0%|          | 0/3676 [00:00<?, ?it/s]

  0%|          | 0/1226 [00:00<?, ?it/s]

  0%|          | 0/2101 [00:00<?, ?it/s]

In [9]:
# Function to convert the bounding boxes to YOLO format and save them
def save_yolo_annotation(row):
    image_path, class_id, output_dir = row['image_path'], row['class_id'], row['output_dir']

    img = cv2.imread(str(image_path))
    if img is None:
        raise ValueError(f"Could not read image from path: {image_path}")

    height, width, _ = img.shape
    label_file = Path(output_dir) / f"{Path(image_path).stem}.txt"

    ymin, xmin, ymax, xmax = row['ymin'], row['xmin'], row['ymax'], row['xmax']

    # Normalize the coordinates
    x_center = (xmin + xmax) / 2 / width
    y_center = (ymin + ymax) / 2 / height
    bbox_width = (xmax - xmin) / width
    bbox_height = (ymax - ymin) / height

    with open(label_file, 'a') as f:
        f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}\n")

# Parallelize the annotation saving process
def process_dataset(dataframe, output_dir):
    dataframe['output_dir'] = output_dir
    with multiprocessing.Pool() as pool:
        list(tqdm(pool.imap(save_yolo_annotation, dataframe.to_dict('records')), total=len(dataframe)))

In [10]:
# Save train and validation labels to their respective dirs
process_dataset(X_train, TRAIN_LABELS_DIR)
process_dataset(X_val, VAL_LABELS_DIR)

# Create a data.yaml file required by YOLO
class_names = train['class'].unique().tolist()
num_classes = len(class_names)

data_yaml = {
    'train': str(TRAIN_IMAGES_DIR),
    'val': str(VAL_IMAGES_DIR),
    'nc': num_classes,
    'names': class_names
}

# Save the data.yaml file
yaml_path = Path('data.yaml')
with open(yaml_path, 'w') as file:
    yaml.dump(data_yaml, file, default_flow_style=False)

/tmp/ipykernel_23/256000492.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['output_dir'] = output_dir


  0%|          | 0/30777 [00:00<?, ?it/s]

/tmp/ipykernel_23/256000492.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['output_dir'] = output_dir


  0%|          | 0/10252 [00:00<?, ?it/s]

In [11]:
model = YOLO("yolov8n.pt")
print(model.device)

100%|██████████| 6.25M/6.25M [00:00<00:00, 117MB/s]

cpu


In [12]:
model.train(
    data='data.yaml',          # Path to the dataset configuration
    epochs=10,                 # Number of epochs
    imgsz=540,               # Image size
    batch=32,                  # Batch size
    device='0',               # Use the first GPU (0 for the first GPU)
    patience=3, # Number of epochs with no improvement after which training will stop
    amp=True,  #Enable Automatic Mixed Precision
    seed=SEED, 
    
)

# Validate the model on the validation seta
model.val(device='0')

Ultralytics 8.3.49 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=10, time=None, patience=3, batch=32, imgsz=540, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=3, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_widt

100%|██████████| 755k/755k [00:00<00:00, 23.0MB/s]
2024-12-11 09:47:33,941	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-12-11 09:47:34,422	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Overriding model.yaml nc=80 with nc=23

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

100%|██████████| 5.35M/5.35M [00:00<00:00, 100MB/s]


AMP: checks passed ✅
WARNING ⚠️ imgsz=[540] must be multiple of max stride 32, updating to [544]


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 20/20 [00:32<00:00,  1.62s/it]


                   all       1226      10252      0.298      0.235      0.168     0.0703
 Pepper_Bacterial_Spot        132        476      0.221      0.147      0.109     0.0291
       Pepper_Fusarium         52        123      0.401      0.501      0.375      0.124
Corn_Cercospora_Leaf_Spot        202       1600      0.405      0.307      0.293      0.109
      Corn_Common_Rust         79        442      0.334      0.321      0.219     0.0826
   Tomato_Early_Blight         56        476      0.305      0.246      0.201      0.089
       Pepper_Septoria         40        697      0.169      0.023     0.0439     0.0153
       Tomato_Septoria        119       1588      0.256       0.23      0.136     0.0432
      Pepper_Leaf_Curl        138        374      0.198      0.144     0.0952     0.0268
    Pepper_Leaf_Mosaic        147        523          0          0     0.0188    0.00439
           Corn_Streak         87        804      0.255      0.553      0.281     0.0934
          Corn_Hea

val: Scanning /kaggle/working/val/labels.cache... 1226 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1226/1226 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 39/39 [00:29<00:00,  1.30it/s]


                   all       1226      10252      0.304      0.234      0.169     0.0705
 Pepper_Bacterial_Spot        132        476       0.22      0.145      0.111     0.0295
       Pepper_Fusarium         52        123       0.41      0.496       0.38      0.126
Corn_Cercospora_Leaf_Spot        202       1600      0.409      0.304      0.293      0.109
      Corn_Common_Rust         79        442      0.342      0.324       0.22     0.0822
   Tomato_Early_Blight         56        476      0.308      0.244      0.201     0.0886
       Pepper_Septoria         40        697      0.167      0.023     0.0435     0.0153
       Tomato_Septoria        119       1588      0.255      0.227      0.137     0.0434
      Pepper_Leaf_Curl        138        374      0.204      0.147     0.0967      0.027
    Pepper_Leaf_Mosaic        147        523          0          0     0.0181    0.00433
           Corn_Streak         87        804      0.258      0.551      0.283     0.0944
          Corn_Hea

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7c7b591badd0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.0420

In [13]:
def process_predictions(results, image_file):
    """
    Process YOLO prediction results and return formatted data.
    """
    confidence_threshold = 0.35
    all_data = []
    filtered_predictions = []
    names = results[0].names

    # Check if predictions exist
    if len(results[0].boxes) == 0:  # No boxes detected
        all_data.append({
            'Image_ID': image_file,
            'class': 'NEG',
            'confidence': 1.0,
            'ymin': 0,
            'xmin': 0,
            'ymax': 0,
            'xmax': 0
        })
    else:
        # Extract boxes, classes, and confidence scores
        boxes = results[0].boxes.xyxy.tolist()  # Convert to list
        classes = results[0].boxes.cls.tolist()
        confidences = results[0].boxes.conf.tolist()

        for box, cls, conf in zip(boxes, classes, confidences):
            if conf >= confidence_threshold:  
                x1, y1, x2, y2 = box
                filtered_predictions.append({
                    'Image_ID': image_file,
                    'class': names[int(cls)],
                    'confidence': conf,
                    'ymin': y1,
                    'xmin': x1,
                    'ymax': y2,
                    'xmax': x2
                })

        # If no predictions meet the threshold, take the highest confidence prediction
        if not filtered_predictions:
            max_index = confidences.index(max(confidences))
            x1, y1, x2, y2 = boxes[max_index]
            detected_class = names[int(classes[max_index])]
            filtered_predictions.append({
                'Image_ID': image_file,
                'class': detected_class,
                'confidence': confidences[max_index],
                'ymin': y1,
                'xmin': x1,
                'ymax': y2,
                'xmax': x2
            })

        # Append filtered predictions to the final data
        all_data.extend(filtered_predictions)

    return all_data

In [14]:
all_data = []
image_files = os.listdir(TEST_IMAGES_DIR)
for image_file in tqdm(image_files):
    img_path = os.path.join(TEST_IMAGES_DIR, image_file)
    results = model(img_path, device='0', augment=True) 
    all_data.extend(process_predictions(results, image_file))

  0%|          | 0/2101 [00:00<?, ?it/s]


image 1/1 /kaggle/working/test/images/id_vugben.jpg: 416x544 (no detections), 115.2ms
Speed: 2.7ms preprocess, 115.2ms inference, 0.7ms postprocess per image at shape (1, 3, 416, 544)

image 1/1 /kaggle/working/test/images/id_c429kf.jpg: 416x544 (no detections), 23.4ms
Speed: 2.6ms preprocess, 23.4ms inference, 0.5ms postprocess per image at shape (1, 3, 416, 544)

image 1/1 /kaggle/working/test/images/id_8dg3gv.jpg: 384x544 9 Tomato_Healthys, 73.2ms
Speed: 1.8ms preprocess, 73.2ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 544)

image 1/1 /kaggle/working/test/images/id_emro8e.jpg: 416x544 1 Pepper_Fusarium, 23.4ms
Speed: 2.5ms preprocess, 23.4ms inference, 3.0ms postprocess per image at shape (1, 3, 416, 544)

image 1/1 /kaggle/working/test/images/id_may8mk.jpg: 416x544 1 Pepper_Leaf_Curl, 23.2ms
Speed: 2.5ms preprocess, 23.2ms inference, 1.3ms postprocess per image at shape (1, 3, 416, 544)

image 1/1 /kaggle/working/test/images/id_q22l7e.jpg: 256x544 1 Corn_Healthy

In [15]:
# Convert the results to a DataFrame and save it
sub = pd.DataFrame(all_data)
sub.to_csv('/kaggle/working/benchmark_submission1.csv', index=False)